In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

shapefile='https://datanova.laposte.fr/explore/dataset/geoflar-departements-2015/download/?format=shp&timezone=Europe/Berlin&lang=fr'
    
gdf = gpd.read_file(shapefile)[['code_dept','nom_dept','geometry','code_reg','nom_reg']]
gdf.sort_values("code_reg",inplace=True)
gdf[["code_reg","nom_reg"]].groupby("code_reg").sum()

In [ ]:
gdf2=gdf.copy()
list_translation={"GUADELOUPE":(63,23),
                  "MARTINIQUE":(63,23),
                  "GUYANE":(50,35),
                  "REUNION":(-51,60),
                  "MAYOTTE":(-38,51.5)}
for d,t in list_translation.items():
    gdf2.loc[gdf2["nom_dept"]==d,"geometry"]=gdf2[gdf2["nom_dept"]==d]["geometry"].translate(t[0],t[1])
gdf2.head(10)

In [ ]:
# cas des tests
# Source https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

import urllib.request, json 
import pandas as pd
from io import StringIO

url_covid_france='https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675' # weekly = 'https://www.data.gouv.fr/fr/datasets/r/dd3ac13c-e87f-4b33-8897-07baff4e1783'

with urllib.request.urlopen(url_covid_france) as url:
    data=pd.read_table(StringIO(url.read().decode()),sep=",")
data.head()

In [ ]:
d=data
d["cases"]=d["p"]
#d["date"]=pd.to_datetime(d['week']+"-6", format='%G-S%V-%w') # weekly case
d["date"]=pd.to_datetime(d['jour'], format='%Y-%m-%d')
d=d[d["cl_age90"]==0]
d.tail()

In [ ]:
import datetime
import pandas as pd

lastdate=max(d["date"])-datetime.timedelta(weeks=1)
#lastdate=datetime.datetime(2020,6,12)

print(lastdate)
prevdate=lastdate-datetime.timedelta(weeks=1)

dsum=d[d["date"]>=prevdate].groupby("dep")[["p","t"]].sum().sort_values("dep")
dsum["rate"]=100.*dsum["p"]/dsum["t"]
dsum.head()

In [ ]:
merged=gdf2.merge(dsum, left_on = 'code_dept', right_on = 'dep', how = 'left')
geojson=merged.to_json()
merged.head(10)

In [ ]:
from bokeh.io import curdoc, output_notebook, show
from bokeh.models import Slider, HoverTool, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.layouts import widgetbox, row, column
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource( geojson=geojson )

#Define a sequential multi-hue color palette.
palette = brewer['Reds'][9]
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5, nan_color = '#d9d9d9')

#Add hover tool
hover = HoverTool(tooltips = [ ('Département','@nom_dept (@code_dept)'),
                              ('Nombre de tests hebdo','@t'),
                              ('Nombre de cas positifs hebdo','@p'),
                              ('Taux positivité','@rate')])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)


#Create figure object.
p = figure(title = 'Taux de positivité intégré sur une semaine, au '+str(lastdate), plot_height = 800 , plot_width = 750, tools = [hover]) # toolbar_location = None,
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'rate', 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

p.add_layout(color_bar, 'below')

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p)#,widgetbox(slider))
#curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(layout)

In [ ]:
dsum[dsum["rate"]<10]["rate"].plot.hist()#(bins=12, alpha=0.5)

In [ ]:
gdf.sort_values("code_dept",inplace=True)
gdf.tail()

In [ ]:
dsum[dsum["rate"]>5]

In [ ]:
merged2 = merged.dissolve(by=['code_reg','nom_reg'], aggfunc='sum')[["geometry","p","t"]]
merged2["rate"]=100.*merged2["p"]/merged2["t"]
merged2.reset_index(inplace=True)
geojson2=merged2.to_json()
merged2.head(10)

In [ ]:
from bokeh.io import curdoc, output_notebook, output_file, show
from bokeh.models import Slider, HoverTool, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.layouts import widgetbox, row, column
from bokeh.palettes import brewer, mpl
from bokeh.plotting import figure
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource( geojson=geojson2 )

#Define a sequential multi-hue color palette.
#palette = brewer['Reds'][9]
palette = mpl['Inferno'][256]
palette = palette[::-1][0:128]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5, nan_color = '#d9d9d9')

#Add hover tool
hover = HoverTool(tooltips = [ ('Région','@nom_reg'),
                              ('Nombre de tests hebdo','@t'),
                              ('Nombre de cas positifs hebdo','@p'),
                              ('Taux positivité','@rate')])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)


#Create figure object.
p = figure(title = 'Taux de positivité intégré sur une semaine, au '+str(lastdate), plot_height = 800 , plot_width = 700, tools = [hover]) # toolbar_location = None,
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'rate', 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

p.add_layout(color_bar, 'below')

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p)#,widgetbox(slider))
#curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()
#output_file("pos_reg_20200612.html")

#Display plot
show(layout)